<a href="https://colab.research.google.com/github/evinracher/3010090-ontological-engineering/blob/main/week2/part1/2_05_Tools_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Tools en LangChain 1.0 con Gemini

Formato: Google Colab

## 1. Configuración de la API Key

In [1]:
from google.colab import userdata
import os

# Obtener la API key desde userdata
api_key = userdata.get('GOOGLE_API_KEY')

# Opcional: Guardarla como variable de entorno
os.environ['GOOGLE_API_KEY'] = api_key

# Verificar que se haya cargado correctamente
print("API Key cargada:", "Sí" if api_key else "No")
print("Primeros caracteres:", api_key[:10] if api_key else "No encontrada")



API Key cargada: Sí
Primeros caracteres: AIzaSyDx4e


## 2. Instalación del entorno

In [2]:
!pip install -q -U langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 5.5 MB/s eta 0:00:00


In [3]:
pip show langchain

Name: langchain
Version: 1.2.8
Summary: Building applications with LLMs through composability
Home-page: https://docs.langchain.com/
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: langchain-core, langgraph, pydantic
Required-by: 


## 3. Importar de librerías

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
import math

## 4. Tool simple: raiz cuadrada

In [5]:
@tool
def calcular_raiz(num: float) -> float:
    """Calcula la raíz cuadrada de un número."""
    return math.sqrt(num)

### 5. Crear e invocar modelo con Tool

In [6]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage, SystemMessage

In [7]:
# Crear el modelo Gemini  OPCION 1
modelo = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0.2,
    model_kwargs={"tools": [calcular_raiz]}, # Langchain NO define tools como parámetro en su constructor, se recomienda esto como solución oficial
)

In [13]:
# Invocar el modelo e imprimir resultado
# ---------------------------------------
respuesta = modelo.invoke("¿Cuánto es la raíz cuadrada de 16?")
print("RESPUESTA:", respuesta)
print("RESPUESTA:", respuesta.tool_calls)

RESPUESTA: content='La raíz cuadrada de 16 es **4**.\n\nEsto se debe a que 4 multiplicado por sí mismo (4 * 4) es igual a 16.' additional_kwargs={} response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019c5cb0-b552-7982-8806-2ed7793094f1-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 12, 'output_tokens': 36, 'total_tokens': 48, 'input_token_details': {'cache_read': 0}}
RESPUESTA: []


## 5. Tool compleja: conversión de moneda

In [9]:
@tool
def usd_to_cop(cantidad_usd: float) -> float:
    '''Convierte USD a COP'''
    return cantidad_usd * 4200

In [10]:
# Crear el modelo Gemini OPCION 2 (sin llamar la Tool usd_to_cop)
modelo = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0.2,
)

In [16]:
respuesta = modelo.invoke("Convierte 150 dólares a COP",tools=[usd_to_cop])

print("🔧 Tool call generada por el modelo:")
print(respuesta)
print(respuesta.tool_calls)

🔧 Tool call generada por el modelo:
content='' additional_kwargs={'function_call': {'name': 'usd_to_cop', 'arguments': '{"cantidad_usd": 150}'}} response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019c5cb4-862b-7613-91ba-46d798c9e403-0' tool_calls=[{'name': 'usd_to_cop', 'args': {'cantidad_usd': 150}, 'id': 'c955bc8e-60e7-42ee-93b8-a45f751f20b2', 'type': 'tool_call'}] invalid_tool_calls=[] usage_metadata={'input_tokens': 53, 'output_tokens': 21, 'total_tokens': 74, 'input_token_details': {'cache_read': 0}}
[{'name': 'usd_to_cop', 'args': {'cantidad_usd': 150}, 'id': 'c955bc8e-60e7-42ee-93b8-a45f751f20b2', 'type': 'tool_call'}]


In [12]:
for call in respuesta.tool_calls:
    if call["name"] == "usd_to_cop":
        final = usd_to_cop.invoke(call["args"])
        print("RESPUESTA:", final)

RESPUESTA: 630000.0
